<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/warp_drive_optimization_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch numpy scikit-learn matplotlib

In [ ]:
#!/usr/bin/env python3
"""
warp_drive_optimization.py

Train a neural network to predict the Alcubierre warp‐drive
energy density proxy over 3D coordinates (x, y, z).
"""

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# 1. Warp Metric Definition
def warp_drive_metric(x, y, z, v=1.0, sigma=1.0):
    """
    Gaussian warp‐bubble proxy: exp(–r²/σ) × v,
    where r = sqrt(x² + y² + z²).
    """
    r2 = x**2 + y**2 + z**2
    return np.exp(-r2 / sigma) * v

# 2. Neural Model
class WarpDriveAI(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=32, output_dim=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)

# 3. Dataset Generation
def generate_dataset(n_samples=15000, v=1.0, sigma=1.0):
    """
    Returns:
      coords: array of shape [n_samples, 3]
      energies: array of shape [n_samples, 1]
    """
    coords = np.random.uniform(-3, 3, size=(n_samples, 3)).astype(np.float32)
    energies = warp_drive_metric(
        coords[:,0], coords[:,1], coords[:,2], v=v, sigma=sigma
    ).reshape(-1, 1).astype(np.float32)
    return coords, energies

# 4. Training Loop
def train_model(
    model, x_train, y_train, x_val, y_val,
    epochs=100, batch_size=256, lr=1e-3
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    train_losses, val_losses = [], []

    for epoch in range(1, epochs + 1):
        # Shuffle and batch training data
        perm = np.random.permutation(len(x_train))
        batch_loss = []

        model.train()
        for i in range(0, len(x_train), batch_size):
            idx = perm[i : i + batch_size]
            xb = torch.from_numpy(x_train[idx]).to(device)
            yb = torch.from_numpy(y_train[idx]).to(device)

            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
            batch_loss.append(loss.item())

        # Validation
        model.eval()
        with torch.no_grad():
            xv = torch.from_numpy(x_val).to(device)
            yv = torch.from_numpy(y_val).to(device)
            val_loss = criterion(model(xv), yv).item()

        train_losses.append(np.mean(batch_loss))
        val_losses.append(val_loss)

        if epoch % 10 == 0 or epoch == 1:
            print(
                f"Epoch {epoch:03d} – "
                f"Train Loss: {train_losses[-1]:.6f} – "
                f"Val Loss: {val_losses[-1]:.6f}"
            )

    return train_losses, val_losses

# 5. Main Execution
if __name__ == "__main__":
    # Hyperparameters
    N_SAMPLES   = 15000
    TEST_SIZE   = 0.2
    HIDDEN_DIM  = 32
    EPOCHS      = 100
    BATCH_SIZE  = 256
    LEARNING_RT = 1e-3

    # Generate and split dataset
    coords, energies = generate_dataset(n_samples=N_SAMPLES)
    x_tr, x_val, y_tr, y_val = train_test_split(
        coords, energies, test_size=TEST_SIZE, random_state=42
    )

    # Initialize model
    model = WarpDriveAI(input_dim=3, hidden_dim=HIDDEN_DIM, output_dim=1)

    # Train and collect losses
    train_losses, val_losses = train_model(
        model, x_tr, y_tr, x_val, y_val,
        epochs=EPOCHS, batch_size=BATCH_SIZE, lr=LEARNING_RT
    )

    # Plot learning curves
    plt.figure(figsize=(6,4))
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses,   label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("MSE Loss")
    plt.title("WarpDriveAI Training Curve")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Evaluate on sample points
    test_pts = np.array([[0,0,0], [1,1,1], [2,0,0]], dtype=np.float32)
    true_vals = warp_drive_metric(
        test_pts[:,0], test_pts[:,1], test_pts[:,2]
    )
    with torch.no_grad():
        preds = model(torch.from_numpy(test_pts))
    print("\nSample Predictions vs True Values:")
    for pt, p, t in zip(test_pts, preds.numpy().flatten(), true_vals.flatten()):
        print(f"Point {pt} → Pred: {p:.4f} | True: {t:.4f}")